In [2]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import dataretrieval
import hydrofunctions
#import pygeohydro
#import ulmo
#import wellapplication

import pastas

In [3]:
os.getcwd()

'/home/mlr/git/hydro-tools'

## hydrofunctions

In [29]:
dir(hydrofunctions)
help(hydrofunctions)
hydrofunctions.__version__

Help on package hydrofunctions:

NAME
    hydrofunctions

DESCRIPTION
    Hydrofunctions
    ~~~~~~~~~~~~~~
    
    Hydrofunctions is a suite of convenience functions to help you explore
    hydrology data interactively.
    
    Basic Usage::
    
        >>> import hydrofunctions as hf
    
        >>> site = '01570500'
        >>> harrisburg = hf.NWIS(site, 'iv', period='P10D')
        Requested data from https://waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=01570500&period=P10D
    
        >>> harrisburg.ok
        True
    
    Examine the dataset::
    
        >>> harrisburg
        USGS:01570500: Susquehanna River at Harrisburg, PA
            00045: <30 * Minutes> Precipitation, total, inches
            00060: <30 * Minutes> Discharge, cubic feet per second
            00065: <30 * Minutes> Gage height, feet
        Start: 2019-04-06 00:30:00+00:00
        End:   2019-04-15 23:00:00+00:00
    
    The listing reports each of the parameters collected at the site th

'0.2.1'

In [30]:
fox_river_gb_dv = hydrofunctions.NWIS("040851385", "dv", period="P10D")
print(fox_river_gb_dv)
print(len(fox_river_gb_dv.df().index))

# this is a handy method to look at the response code, maybe useful
print(fox_river_gb_dv.response)

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=040851385&period=P10D
USGS:040851385: FOX RIVER AT OIL TANK DEPOT AT GREEN BAY, WI
    00010: <Day>  Temperature, water, degrees Celsius 
    00060: <Day>  Discharge, cubic feet per second 
    00095: <Day>  Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius 
    00300: <Day>  Dissolved oxygen, water, unfiltered, milligrams per liter 
    00400: <Day>  pH, water, unfiltered, field, standard units 
    63680: <Day>  Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU) 
Start: 2021-10-18 00:00:00+00:00
End:   2021-10-28 00:00:00+00:00
11
<Response [200]>


In [31]:
fox_river_gb_iv = hydrofunctions.NWIS("040851385", "iv", period="P10D")
print(fox_river_gb_iv)
print(len(fox_river_gb_iv.df().index))
print(fox_river_gb_iv.response)

Requested data from https://waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=040851385&period=P10D
USGS:040851385: FOX RIVER AT OIL TANK DEPOT AT GREEN BAY, WI
    00010: <5 * Minutes>  Temperature, water, degrees Celsius [YSI]
    00060: <5 * Minutes>  Discharge, cubic feet per second [(2)]
    00065: <5 * Minutes>  Gage height, feet [PS-2]
    00095: <5 * Minutes>  Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius [YSI]
    00300: <5 * Minutes>  Dissolved oxygen, water, unfiltered, milligrams per liter [YSI]
    00400: <5 * Minutes>  pH, water, unfiltered, field, standard units [YSI]
    63680: <5 * Minutes>  Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU) [YSI 6136]
    72255: <5 * Minutes>  Mean water velocity for discharge computation, feet per second [X SONTEK]
Start: 2021-10-18 15:10:00+00:00
End:   2021-10-28 14:20:00+00:00
2871
<Re

In [33]:
fox_river_gb_iv = hydrofunctions.NWIS("040851385", "iv", "2021-10-15", "2021-10-25")
print(fox_river_gb_iv)
print(len(fox_river_gb_iv.df().index))
print(fox_river_gb_iv.response)

Requested data from https://waterservices.usgs.gov/nwis/iv/?format=json%2C1.1&sites=040851385&startDT=2021-10-15&endDT=2021-10-25
USGS:040851385: FOX RIVER AT OIL TANK DEPOT AT GREEN BAY, WI
    00010: <5 * Minutes>  Temperature, water, degrees Celsius [YSI]
    00060: <5 * Minutes>  Discharge, cubic feet per second [(2)]
    00065: <5 * Minutes>  Gage height, feet [PS-2]
    00095: <5 * Minutes>  Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius [YSI]
    00300: <5 * Minutes>  Dissolved oxygen, water, unfiltered, milligrams per liter [YSI]
    00400: <5 * Minutes>  pH, water, unfiltered, field, standard units [YSI]
    63680: <5 * Minutes>  Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU) [YSI 6136]
    72255: <5 * Minutes>  Mean water velocity for discharge computation, feet per second [X SONTEK]
Start: 2021-10-15 05:00:00+00:00
End:   2021-10-26

In [34]:
fox_river_gb_iv.df()

,USGS:040851385:00010:00000,USGS:040851385:00010:00000_qualifiers,USGS:040851385:00060:00000,USGS:040851385:00060:00000_qualifiers,USGS:040851385:00065:00000,USGS:040851385:00065:00000_qualifiers,USGS:040851385:00095:00000,USGS:040851385:00095:00000_qualifiers,USGS:040851385:00300:00000,USGS:040851385:00300:00000_qualifiers,USGS:040851385:00400:00000,USGS:040851385:00400:00000_qualifiers,USGS:040851385:63680:00000,USGS:040851385:63680:00000_qualifiers,USGS:040851385:72255:00000,USGS:040851385:72255:00000_qualifiers
datetimeUTC,,,,,,,,,,,,,,,,
2021-10-15 05:00:00+00:00,18.8,P,1250.0,P,579.96,P,366.0,P,6.6,P,8.1,P,22.8,P,0.09,P
2021-10-15 05:05:00+00:00,18.7,P,1800.0,P,579.97,P,367.0,P,6.8,P,8.1,P,19.8,P,0.13,P
2021-10-15 05:10:00+00:00,18.7,P,2400.0,P,579.98,P,367.0,P,6.8,P,8.1,P,19.3,P,0.17,P
2021-10-15 05:15:00+00:00,18.7,P,1670.0,P,580.00,P,367.0,P,6.7,P,8.1,P,18.5,P,0.12,P
2021-10-15 05:20:00+00:00,18.7,P,1760.0,P,580.02,P,367.0,P,6.8,P,8.1,P,18.3,P,0.13,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-26 04:35:00+00:00,12.3,P,8410.0,P,581.12,P,378.0,P,8.8,P,8.2,P,13.1,P,0.58,P
2021-10-26 04:40:00+00:00,12.3,P,8040.0,P,581.11,P,378.0,P,8.7,P,8.2,P,10.9,P,0.55,P
2021-10-26 04:45:00+00:00,12.3,P,9370.0,P,581.09,P,377.0,P,8.8,P,8.2,P,11.8,P,0.64,P


In [17]:
wi_code = "WI"
wi = hydrofunctions.NWIS(stateCd=wi_code)
type(wi)

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&stateCd=WI


hydrofunctions.station.NWIS

In [ ]:
dir(wi)

## python dataretrieval

In [24]:
help(dataretrieval)
print(dir(dataretrieval))
#dataretrieval.__version__

Help on package dataretrieval:

NAME
    dataretrieval

PACKAGE CONTENTS
    codes (package)
    nadp
    nwis
    streamstats
    utils
    wqp

FILE
    /home/mlr/env/hydro/lib/python3.8/site-packages/dataretrieval/__init__.py


['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'codes', 'nwis', 'utils']


In [ ]:
# dataretrivel is broken into modules, need to import individual pieaces as needed
from dataretrieval import nwis, utils

# refernence gage, time, and data
fox_river_gb = "040851385"
start = "2021-10-10"
end = "2021-10-20"
data = ["iv", "dv", "qwdata", "site"]

# attempt to get iv, dv, site, and qwdata... raise error if data does not exist
for d in data:
    try:
        nwis.get_record(sites=fox_river_gb, service=d, start=start, end=end).info()
        print("-"*20)

    except utils.NoSitesError as e:
        print(f"{d}: {e}")
        print("-"*20)

In [20]:
# use specific dataretrieval functions to get data, these follow R dataRetrieval style
from dataretrieval import nwis

# refernence gage, time, and data
fox_river_gb = "040851385"
start = "2021-10-15"
end = "2021-10-25"

# returns a tuple with dataframe and metadata class
dv = nwis.get_dv(start=start, end=end, sites=fox_river_gb)
iv = nwis.get_iv(start=start, end=end, sites=fox_river_gb)
info = nwis.get_info(sites=fox_river_gb)

In [21]:
type(dv)
for i in dv:
    print(type(i))

<class 'pandas.core.frame.DataFrame'>
<class 'dataretrieval.utils.Metadata'>


In [22]:
type(iv)
for i in dv:
    print(type(i))

<class 'pandas.core.frame.DataFrame'>
<class 'dataretrieval.utils.Metadata'>


In [24]:
type(info)
for i in info:
    print(type(i))

<class 'pandas.core.frame.DataFrame'>
<class 'dataretrieval.utils.Metadata'>


In [26]:
iv[0].head()

,00010_ysi,00010_ysi_cd,site_no,00060_2,00060_2_cd,00065_ps-2,00065_ps-2_cd,00095_ysi,00095_ysi_cd,00300_ysi,00300_ysi_cd,00400_ysi,00400_ysi_cd,63680_ysi 6136,63680_ysi 6136_cd,72255_x sontek,72255_x sontek_cd
datetime,,,,,,,,,,,,,,,,,
2021-10-15 00:00:00-05:00,18.8,P,040851385,1250.0,P,579.96,P,366.0,P,6.6,P,8.1,P,22.8,P,0.09,P
2021-10-15 00:05:00-05:00,18.7,P,040851385,1800.0,P,579.97,P,367.0,P,6.8,P,8.1,P,19.8,P,0.13,P
2021-10-15 00:10:00-05:00,18.7,P,040851385,2400.0,P,579.98,P,367.0,P,6.8,P,8.1,P,19.3,P,0.17,P
2021-10-15 00:15:00-05:00,18.7,P,040851385,1670.0,P,580.00,P,367.0,P,6.7,P,8.1,P,18.5,P,0.12,P
2021-10-15 00:20:00-05:00,18.7,P,040851385,1760.0,P,580.02,P,367.0,P,6.8,P,8.1,P,18.3,P,0.13,P


In [58]:
# pull out info dataframe and transpose
df_info = info[0]
df_info.T.head()

,0
agency_cd,USGS
site_no,040851385
station_nm,"FOX RIVER AT OIL TANK DEPOT AT GREEN BAY, WI"
site_tp_cd,ST
lat_va,443143


In [60]:
# pull out metadata class and look at attributes, can't remember a better way to print
meta_info = info[1]
print(meta_info.comment)
print(meta_info.disclaimer)
print(meta_info.header)
print(meta_info.query_time)
print(meta_info.site_info)
print(meta_info.statistic_info)
print(meta_info.url)
print(meta_info.variable_info)



 US Geological Survey
 retrieved: 2021-10-22 17:02:00 -04:00	(vaas01)

 The Site File stores location and general information about groundwater,
 surface water, and meteorological sites
 for sites in USA.

 File-format description:  http://help.waterdata.usgs.gov/faq/about-tab-delimited-output
 Automated-retrieval info: http://waterservices.usgs.gov/rest/Site-Service.html

 Contact:   gs-w_support_nwisweb@usgs.gov

 The following selected fields are included in this output:

  agency_cd       -- Agency
  site_no         -- Site identification number
  station_nm      -- Site name
  site_tp_cd      -- Site type
  lat_va          -- DMS latitude
  long_va         -- DMS longitude
  dec_lat_va      -- Decimal latitude
  dec_long_va     -- Decimal longitude
  coord_meth_cd   -- Latitude-longitude method
  coord_acy_cd    -- Latitude-longitude accuracy
  coord_datum_cd  -- Latitude-longitude datum
  dec_coord_datum_cd -- Decimal Latitude-longitude datum
  district_cd     -- District code


In [9]:
from dataretrieval import nwis
codes = nwis.get_pmcodes()
codes

(                                                                                parameter_cd  \
 32001 Information Sample volume, gallons                             NaN                 NaN   
 45700 Information Gate openings, reservoir, all gates, feet          NaN                 NaN   
 49275 Information alpha-HCH-d6, surrogate, bed sediment smaller t... 86194-41-4          NaN   
 49276 Information PCB congener 204, surrogate, bed sediment small... 74472-52-9          NaN   
 49277 Information PCB congener 14, surrogate, bed sediment smalle... 34883-41-5          NaN   
 ...                                                                                      ...   
 80290 Sediment    Total sediment, fall diameter (native water), p... NaN            Sediment   
 80291 Sediment    Total sediment, fall diameter (native water), p... NaN            Sediment   
 80292 Sediment    Total sediment, fall diameter (native water), p... NaN            Sediment   
 80293 Sediment    Bed sedimen

In [ ]:
# hydrofunctions and dataretrieval implement datetime indexes differently
print(hf_data.index)
print(dt_data.index)

## test returned data

looking at data returned by hydrofunctions and dataretrieval from a fixed stream gage. how close are the timeseries for a given channel? do the date time indexes align?

In [1]:
import pandas as pd

import dataretrieval
import hydrofunctions

from dataretrieval import nwis

# parameter codes
temp_c = "00010"       # temp celsius
discharge_q = "00060"  # cubic feet per second
gage_height = "00065"  # height

def get_data(gage, data, start, end):
    '''
    helper to get stream gage iv discharge data over NWIS with dataretrieval and hydrofunctions.
    
    returns hydrofunctions dataframe and dataretrieval dataframe.
    '''
    
    # retrieve data with hydrofunctions, get dataframe of discharge, convert datetime to CDT
    hf_data = hydrofunctions.NWIS(fox_river_gb, data, start, end).df("discharge").tz_convert('US/Central')
    
    # retrieve data with dataretrieval, query with discharge parameter code, get dataframe tuple
    dt_data = nwis.get_iv(start=start, end=end, sites=fox_river_gb, parameterCd="00060")[0]
    
    return hf_data, dt_data

In [2]:
# refernence gage, time, and data
fox_river_gb = "040851385"
data = "iv"
start = "2021-10-15"
end = "2021-10-25"

hf_df, dt_df = get_data(gage=fox_river_gb, data=data, start=start, end=end)

In [62]:
print(f"hydrofunctions {data} data from {start} to {end}: {len(hf_df.index)} observations")
print(f"dataretrieval {data} data from {start} to {end}: {len(dt_df.index)} observations")
print("-"*50)

# figure out how many records there should be over N days with measurements taken at 15 minute intervals

# hydrofunctions
hf_length = len(hf_df.index)

# dataretrieval
dt_length = len(dt_df.index)

# how many 5 minute observations from 2021-10-15 to 2021-10-25 (10 days)
perhour = 60/5
perday = 24*perhour
perperiod = 10*perday
print(f"observations: 10days = {perperiod}, 1day= {perday}, 1hour= {perhour}")
print("-"*50)

# observation differences
hf_diff = hf_length-perperiod
dt_diff = dt_length-perperiod

def text(diff):
    return "extra hours of observations" if diff >= 0 else "missing hours of observations"

def hours(diff):
    return abs(round(diff/perhour, 2))

print(f"hydrofunctions difference: {abs(hf_diff)} observations, {hours(hf_diff)} {text(hf_diff)}")
print(f"python-dataretrieval difference: {abs(dt_diff)} observations, {hours(dt_diff)} {text(dt_diff)}")

hydrofunctions iv data from 2021-10-15 to 2021-10-25: 3168 observations
dataretrieval iv data from 2021-10-15 to 2021-10-25: 2782 observations
--------------------------------------------------
observations: 10days = 2880.0, 1day= 288.0, 1hour= 12.0
--------------------------------------------------
hydrofunctions difference: 288.0 observations, 24.0 extra hours of observations
python-dataretrieval difference: 98.0 observations, 8.17 missing hours of observations


In [65]:
# need to find where time is missing in each dataframe. there are no nans. do i need to identify jumps/gaps in the time index?

from datetime import datetime, timedelta
import pandas as pd

# Construct dummy dataframe
dates = pd.to_datetime([
    '2016-08-03',
    '2016-08-04',
    '2016-08-05',
    '2016-08-17',
    '2016-09-05',
    '2016-09-06',
    '2016-09-07',
    '2016-09-19'])
df = pd.DataFrame(dates, columns=['date'])

# Take the diff of the first column (drop 1st row since it's undefined)
deltas = df['date'].diff()[1:]

# Filter diffs (here days > 1, but could be seconds, hours, etc)
gaps = deltas[deltas > timedelta(days=1)]

# Print results
print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}')
for i, g in gaps.iteritems():
    gap_start = df['date'][i - 1]
    print(f'Start: {datetime.strftime(gap_start, "%Y-%m-%d")} | '
          f'Duration: {str(g.to_pytimedelta())}')
    
dt_df.index.diff()

3 gaps with average gap duration: 14 days 08:00:00
Start: 2016-08-05 | Duration: 12 days, 0:00:00
Start: 2016-08-17 | Duration: 19 days, 0:00:00
Start: 2016-09-07 | Duration: 12 days, 0:00:00


AttributeError: 'DatetimeIndex' object has no attribute 'diff'

In [10]:
# look at dataframe info
print(hf_df.info())
print("-*"*40)
print(dt_df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3168 entries, 2021-10-15 00:00:00-05:00 to 2021-10-25 23:55:00-05:00
Freq: 5T
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   USGS:040851385:00060:00000  3168 non-null   float64
dtypes: float64(1)
memory usage: 178.5 KB
None
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2782 entries, 2021-10-15 00:00:00-05:00 to 2021-10-25 23:55:00-05:00
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   00060_2     2782 non-null   float64
 1   00060_2_cd  2782 non-null   object 
 2   site_no     2782 non-null   object 
dtypes: float64(1), object(2)
memory usage: 86.9+ KB
None


In [13]:
hf_df.head()

,USGS:040851385:00060:00000
datetimeUTC,
2021-10-15 00:00:00-05:00,1250.0
2021-10-15 00:05:00-05:00,1800.0
2021-10-15 00:10:00-05:00,2400.0
2021-10-15 00:15:00-05:00,1670.0
2021-10-15 00:20:00-05:00,1760.0


In [14]:
dt_df.head()

,00060_2,00060_2_cd,site_no
datetime,,,
2021-10-15 00:00:00-05:00,1250.0,P,040851385
2021-10-15 00:05:00-05:00,1800.0,P,040851385
2021-10-15 00:10:00-05:00,2400.0,P,040851385
2021-10-15 00:15:00-05:00,1670.0,P,040851385
2021-10-15 00:20:00-05:00,1760.0,P,040851385


In [9]:
# look at first time step
print(hf_df.index[1], hf_df.iloc[0].tolist())
print(dt_df.index[1], dt_df.iloc[0].tolist())

# look at last time step
print(hf_df.index[-1], hf_df.iloc[-1].tolist())
print(dt_df.index[-1], dt_df.iloc[-1].tolist())

2021-10-15 00:05:00-05:00 [1250.0]
2021-10-15 00:05:00-05:00 [1250.0, 'P', '040851385']
2021-10-25 23:55:00-05:00 [8140.0]
2021-10-25 23:55:00-05:00 [8140.0, 'P', '040851385']


In [43]:
print("parameter code:",dt_df.columns[1])
dt_df.iloc[:,1]

parameter code: 00060_2_cd


datetime
2021-10-15 00:00:00-05:00    P
2021-10-15 00:05:00-05:00    P
2021-10-15 00:10:00-05:00    P
2021-10-15 00:15:00-05:00    P
2021-10-15 00:20:00-05:00    P
                            ..
2021-10-25 23:35:00-05:00    P
2021-10-25 23:40:00-05:00    P
2021-10-25 23:45:00-05:00    P
2021-10-25 23:50:00-05:00    P
2021-10-25 23:55:00-05:00    P
Name: 00060_2_cd, Length: 2780, dtype: object

In [44]:
dt_df.iloc[:,1].unique()

array(['P'], dtype=object)